# **Reti Neurali** 

## **Esercizio 1: Download e pre-processamento dei dati.**

Scaricare e pre-processare i dati per il successivo addestramento del modello. 

Il dataset che utilizzeremo sarà CIFAR10, scaricabile dalla libreria `tensorflow.keras.datasets`

In [18]:
!pip install tensorflow

In [19]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

# Download dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = y_train.ravel()
y_test = y_test.ravel()

# Stampare le shape dei dati
print(y_train.shape, y_test.shape, x_train.shape, x_test.shape)

(50000,) (10000,) (50000, 32, 32, 3) (10000, 32, 32, 3)


In [20]:
from sklearn.preprocessing import StandardScaler

# Pre-processamento dei dati

# svolgimento...
scaler = StandardScaler()
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)
                            
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

train_set = scaler.fit_transform(x_train_flat)
test_set = scaler.transform(x_test_flat)

## **Esercizio 2: Creare modello MLP**

Per creare il modello MLP utilizziamo l' oggetto `MLPClassifier` dal modulo `sklearn.neural_networks`. Questo è un oggetto molto complesso che prevede la possibilità di specificare tanti parametri, permettendoci una personalizzazione molto dettagliata. Vediamo di seguito gli argomenti principali:

- `hidden_layer_sizes`: rappresenta la struttura dell' MLP, sotto forma di una tupla. La tupla deve essere composta da numeri interi, ogni numero indica il numero di neuroni presenti nel rispettivo layer.

Esempio: 

`hidden_layer_sizes` = `(100)`

creerà un solo layer con 100 neuroni

`hidden_layer_sizes` = `(100, 50)`

creerà due layer, il primo con 100 neuroni, il secondo invece con 50.

- `max_iter`: massimo numero di iterazioni per raggiungere la convergenza. 

- `activation`: indica quale funzione di attivazione utilizzare, valori possibili sono `'relu'`, `'logistic'`, `'tanh'` and `'identity'`.

- `solver`: indica quale algoritmo di ottimizzazione utilizzare, valori possibili sono `'adam'`, `'sgd'` and `'lbfgs'`.

- `learning_rate_init`: valore iniziale del learning rate.

- `verbose`: valore booleano che, se impostato su `True`, stampa l' output di ogni iterazione di training. Molto utile per monitorare il training.

- `random_state`: fissa il seed della randomizzazione.


Per iniziare creiamo un MLP molto basilare, alleniamolo e testiamone le performance. Come parametri utilizzeremo:

- `hidden_layer_sizes` = `(100)`

- `max_iter` = `20`

- `random_state` = `42`

In [22]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Creare MLP 

# svolgimento...
MLP = MLPClassifier(hidden_layer_sizes = (100), max_iter = 20, random_state = 42)

# Allenare MLP

# svolgimento...
MLP.fit(train_set, y_train)

# Valutare MLP

# svolgimento...
MLP_pred = MLP.predict(test_set)

# Stampare l' accuratezza

# svolgimento...
MLP_val = accuracy_score(y_test, MLP_pred)
print(MLP_val)

0.5057


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


## **Esercizio 2.1: Aumentiamo i parametri del nostro modello**

Proviamo adesso ad aumentare i dettagli del nostro modello, modificando o aggiungendo i parametri sopra specificati. 

In [24]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Creare MLP con più strati e altre specifiche

# svolgimento...
MLP = MLPClassifier(hidden_layer_sizes = (100, 100, 50), activation = ('logistic'), solver =('sgd'), max_iter = 20, random_state = 60, verbose = True)

# Allenare MLP

# svolgimento...
MLP.fit(train_set, y_train)

# Valutare MLP

# svolgimento...
MLP_pred = MLP.predict(test_set)

# Stampare l' accuratezza

# svolgimento...
MLP_val = accuracy_score(y_test, MLP_pred)
print(MLP_val)

Iteration 1, loss = 2.31130636
Iteration 2, loss = 2.30118435
Iteration 3, loss = 2.30032753
Iteration 4, loss = 2.29942923
Iteration 5, loss = 2.29849973
Iteration 6, loss = 2.29740945
Iteration 7, loss = 2.29631831
Iteration 8, loss = 2.29505662
Iteration 9, loss = 2.29356617
Iteration 10, loss = 2.29192169
Iteration 11, loss = 2.28986605
Iteration 12, loss = 2.28753870
Iteration 13, loss = 2.28459650
Iteration 14, loss = 2.28103278
Iteration 15, loss = 2.27666256
Iteration 16, loss = 2.27115783
Iteration 17, loss = 2.26419713
Iteration 18, loss = 2.25551210
Iteration 19, loss = 2.24471332
Iteration 20, loss = 2.23162823
0.1975


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


## **Esercizio 3: Implementare manualmente l' algoritmo di early stopping.**

L' algoritmo di early stopping ci permette di terminare anticipatamente l' allenamento di un modello nel caso in cui questo raggiunga la convergenza. Supponiamo infatti che il nostro modello raggiunga un certo livello di accuratezza e che non riesca a migliorare oltre quel livello. Questo significa che il modello, da quel momento in poi, non sta più apprendendo nuove informazioni, per cui le successive iterazioni sono superflue, ed inoltre rischiano di essere dannose, spingendo il modello verso l' overfitting. 

L' early stopping verifica ad ogni iterazione che l' accuratezza del modello sia incrementata di una certa tolleranza. Se questa tolleranza non viene superata per un certo numero di epoche, allora possiamo decidere di stoppare l' allenamento in quanto il modello ha raggiunto la convergenza.

**N.B: per applicare early stopping è necessario specificare i seguenti parametri dell' MLP:**

- `warm_start`=`True` in modo che il training proceda dallo stato attuale del modello e non dall' inizializzazione.

- `max_iter`=`1` in modo che il modello venga allenato per una sola epoca. Per l' early stopping infatti dovremo gestire manualmente il numero di iterazioni.

In [29]:
import numpy as np
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import accuracy_score 


n_total_epochs = 100  
patience = 10         
tolerance = 1e-4      

best_test_accuracy = 0.0
epochs_without_improvement = 0

#Creo il mlp
MLP = MLPClassifier(hidden_layer_sizes = (100), max_iter = 1, random_state = 42, warm_start=True)

#alleno con early stopping
for epoch in range(n_total_epochs):
    MLP.fit(train_set, y_train)

     # Valutazione
    MLP_pred = MLP.predict(test_set)
    accuracy = accuracy_score(y_test, MLP_pred)
    print(f"Ciclo {epoch + 1}, Accuratezza: {accuracy:.4f}")

    # Controllo miglioramento
    if accuracy > best_test_accuracy + tolerance:
        best_test_accuracy = accuracy
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print(f"Early stopping attivato dopo: {epoch + 1}")
        break

print("Accuratezza migliore:", best_test_accuracy)

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


Ciclo 1, Accuratezza: 0.4325
Ciclo 2, Accuratezza: 0.4658
Ciclo 3, Accuratezza: 0.4780
Ciclo 4, Accuratezza: 0.4884
Ciclo 5, Accuratezza: 0.4901
Ciclo 6, Accuratezza: 0.4899
Ciclo 7, Accuratezza: 0.4965
Ciclo 8, Accuratezza: 0.4942
Ciclo 9, Accuratezza: 0.4967
Ciclo 10, Accuratezza: 0.4912
Ciclo 11, Accuratezza: 0.4923
Ciclo 12, Accuratezza: 0.4930
Ciclo 13, Accuratezza: 0.4948
Ciclo 14, Accuratezza: 0.4945
Ciclo 15, Accuratezza: 0.4957
Ciclo 16, Accuratezza: 0.4934
Ciclo 17, Accuratezza: 0.4946
Ciclo 18, Accuratezza: 0.4930
Ciclo 19, Accuratezza: 0.4939
Early stopping attivato in: 19
Accuratezza migliore: 0.4967
